In [ ]:
# 1. Install dependencies (Uncomment if running for the first time)
# !pip install -r requirements.txt

import os
import torch
import matplotlib.pyplot as plt
from PIL import Image

# Check device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Running on: {device}")

# Create necessary directories
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

In [ ]:
print("Starting DDPM Training (Pixel Space)...")
# We use the script to keep memory clean
!python train_ddpm.py

In [ ]:
print("Starting Stable Diffusion Training (Latent Space)...")
!python train_sd.py

In [ ]:
# Display the comparison grid generated by the script
if os.path.exists("i2i_comparison.png"):
    img = Image.open("i2i_comparison.png")
    plt.figure(figsize=(20, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
else:
    print("Comparison image not found. Check if compare_inference.py ran successfully.")

In [ ]:
print("To see the detailed timing and loss metrics, run this in your terminal:")
print("mlflow ui")